In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import FamaMacBeth

In [2]:
import linearmodels
linearmodels.__version__

'6.1'

In [5]:
%%time
Cret = pd.read_csv('Data/Cret.csv', index_col=0)
Lret = pd.read_csv('Data/Lret.csv', index_col=0)
RetDiff = pd.read_csv('Data/RetDiff.csv', index_col=0)
MktCap = pd.read_csv('Data/MktCap.csv', index_col=0)
MktCap = np.log(MktCap)
# factors = pd.read_csv('factors.csv', index_col='Index')


Cret.index = pd.to_datetime(Cret.index)
Lret.index = pd.to_datetime(Lret.index)
RetDiff.index = pd.to_datetime(RetDiff.index)
MktCap.index = pd.to_datetime(MktCap.index)
# factors.index = pd.to_datetime(factors.index)


Cret = Cret.loc['2012/02':]
Lret = Lret.loc['2012/02':]
RetDiff = RetDiff.loc['2012/02':]
MktCap = MktCap.loc['2012/02':]
# factors = factors.loc['1951-07':]

CPU times: total: 1 s
Wall time: 1.02 s


In [6]:
MktCap.head()

,X600001,X600004,X600005,X600006,X600007,X600008,X600009,X600010,X600011,X600012,...,X301128,X301129,X301133,X301149,X301155,X301169,X301178,X301180,X301185,X301188
Date,,,,,,,,,,,,,,,,,,,,,
2012-02-29,NaN,15.867774,17.287451,15.859325,16.129344,16.312346,16.479010,17.335627,17.817374,15.490446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-03-31,NaN,15.863332,17.178252,15.702580,16.146134,16.197276,16.463649,17.458887,17.747336,15.468370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-04-30,NaN,15.909718,17.202432,15.755690,16.217931,16.294064,16.466740,17.472243,17.785802,15.526891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-05-31,NaN,15.913958,17.185220,15.767119,16.199531,16.314156,16.474427,17.520796,17.813711,15.477259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-06-30,NaN,15.873665,17.116971,15.656060,16.164573,16.191160,16.449619,17.260641,17.982176,15.413310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
%%time
# set x (lagged variable)
multi_cret = Cret.iloc[:-1].stack()
multi_lret = Lret.iloc[:-1].stack()
multi_retdiff = RetDiff[:-1].stack()
multi_mktcap = MktCap[:-1].stack()

# set y (next-month ret)
ret = Lret.iloc[1:]
ret.index = Lret.iloc[:-1].index
multi_ret  = ret.stack()


multi_cret.rename('Cret', inplace=True)
multi_cret.rename_axis(["Date",'Code'],inplace=True)
multi_lret.rename('Lret', inplace=True)
multi_lret.rename_axis(["Date",'Code'],inplace=True)
multi_retdiff.rename('RetDiff', inplace=True)
multi_retdiff.rename_axis(["Date",'Code'],inplace=True)
multi_mktcap.rename('MktCap', inplace=True)
multi_mktcap.rename_axis(["Date",'Code'],inplace=True)
multi_ret.rename('Ret_Next', inplace=True)
multi_ret.rename_axis(["Date",'Code'],inplace=True)


multi_cret = pd.DataFrame(multi_cret)
multi_lret = pd.DataFrame(multi_lret)
multi_retdiff = pd.DataFrame(multi_retdiff)
multi_mktcap = pd.DataFrame(multi_mktcap)
multi_ret  = pd.DataFrame(multi_ret)


df = multi_cret.join(multi_lret).join(multi_retdiff).join(multi_mktcap).join(multi_ret)

CPU times: total: 312 ms
Wall time: 328 ms


In [8]:
multi_cret.head()

Cret
Date       Code             
2012-02-29 X600004  0.052158
           X600005  0.091296
           X600006  0.112744
           X600007  0.098728
           X600008  0.103561

In [23]:
df.head()

Cret      Lret   RetDiff     MktCap  Ret_Next
Date       Code                                                      
2012-02-29 X600004  0.052158  0.047988 -0.008868  15.867774 -0.004431
           X600005  0.091296  0.059801  0.040118  17.287451 -0.103448
           X600006  0.112744  0.206250 -0.093782  15.859325 -0.145078
           X600007  0.098728  0.097268 -0.005037  16.129344  0.016932
           X600008  0.103561  0.057471  0.049847  16.312346 -0.108696

In [9]:
df = df.reset_index().set_index(['Code','Date'])

In [11]:
df.head()

,,Cret,Lret,RetDiff,MktCap,Ret_Next
Code,Date,,,,,
X600004,2012-02-29,0.052158,0.047988,-0.008868,15.867774,-0.004431
X600005,2012-02-29,0.091296,0.059801,0.040118,17.287451,-0.103448
X600006,2012-02-29,0.112744,0.206250,-0.093782,15.859325,-0.145078
X600007,2012-02-29,0.098728,0.097268,-0.005037,16.129344,0.016932
X600008,2012-02-29,0.103561,0.057471,0.049847,16.312346,-0.108696


In [12]:
mod = FamaMacBeth.from_formula('Ret_Next ~ 1 + Cret + Lret', data=df)
res = mod.fit(cov_type='kernel', debiased=False,bandwidth=6)
print(res.summary)

c:\Users\FanZ\anaconda3\Lib\site-packages\linearmodels\panel\model.py:3057: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


                            FamaMacBeth Estimation Summary                           
Dep. Variable:                    Ret_Next   R-squared:                        0.0008
Estimator:                     FamaMacBeth   R-squared (Between):             -0.0583
No. Observations:                   215600   R-squared (Within):               0.0013
Date:                     Mon, Dec 15 2025   R-squared (Overall):              0.0008
Time:                             15:16:17   Log-likelihood                 1.366e+05
Cov. Estimator:    Fama-MacBeth Kernel Cov                                           
                                             F-statistic:                      84.566
Entities:                             3144   P-value                           0.0000
Avg Obs:                            68.575   Distribution:                F(2,215597)
Min Obs:                            1.0000                                           
Max Obs:                            140.00   F-statist

In [13]:
%%time
idx = Cret.index

numCret = np.zeros(len(idx))
for i in range(len(idx)):
    temp = Cret.loc[idx[i]].dropna()
    numCret[i] = len(temp)
    
numLret = np.zeros(len(idx))
for i in range(len(idx)):
    temp = Lret.loc[idx[i]].dropna()
    numLret[i] = len(temp)

retIdx = ret.index
numRet = np.zeros(len(retIdx))
for i in range(len(retIdx)):
    temp = ret.loc[retIdx[i]].dropna()
    numRet[i] = len(temp)

CPU times: total: 78.1 ms
Wall time: 87 ms


In [14]:
print(np.mean(numCret))
print(np.mean(numLret))
print(np.mean(numRet))

1640.86013986014
1605.7272727272727
1609.8239436619717


### 移除有NA的行（observation）

In [15]:
df_dropna = df.dropna()
mod = FamaMacBeth.from_formula('Ret_Next ~ 1 + Cret + Lret', data=df_dropna)
res = mod.fit(cov_type='kernel', debiased=False, bandwidth=6)
print(res.summary)

                            FamaMacBeth Estimation Summary                           
Dep. Variable:                    Ret_Next   R-squared:                        0.0006
Estimator:                     FamaMacBeth   R-squared (Between):             -0.0548
No. Observations:                   141428   R-squared (Within):               0.0013
Date:                     Mon, Dec 15 2025   R-squared (Overall):              0.0006
Time:                             15:16:27   Log-likelihood                 9.242e+04
Cov. Estimator:    Fama-MacBeth Kernel Cov                                           
                                             F-statistic:                      42.790
Entities:                             2940   P-value                           0.0000
Avg Obs:                            48.105   Distribution:                F(2,141425)
Min Obs:                            1.0000                                           
Max Obs:                            140.00   F-statist

### 将dataframe按照Code排序

In [16]:
df_dropna = df_dropna.sort_index(level=0)

In [31]:
mod = FamaMacBeth.from_formula('Ret_Next ~ 1 + Cret + Lret', data=df_dropna)
res = mod.fit(cov_type='kernel',debiased=False,bandwidth=6)
print(res.summary)

                            FamaMacBeth Estimation Summary                           
Dep. Variable:                    Ret_Next   R-squared:                        0.0006
Estimator:                     FamaMacBeth   R-squared (Between):             -0.0548
No. Observations:                   141428   R-squared (Within):               0.0013
Date:                     Mon, Aug 25 2025   R-squared (Overall):              0.0006
Time:                             13:30:59   Log-likelihood                 9.242e+04
Cov. Estimator:    Fama-MacBeth Kernel Cov                                           
                                             F-statistic:                      42.790
Entities:                             2940   P-value                           0.0000
Avg Obs:                            48.105   Distribution:                F(2,141425)
Min Obs:                            1.0000                                           
Max Obs:                            140.00   F-statist

### 调整输出格式

In [37]:
def FamaMacBeth_summary(DF,
                        reg_lst,
                        reg_order,
                        reg_names=None,
                        params_format='{:.4f}',
                        tvalues_format='{:.4f}'):

    '''
    A function for Fama-MacBeth regression and results summary.

    Parameters
    ----------
    DF: DataFrame
        A panel date of which multi-index is stock and month (datetime64[ns]),
        containing all the dependent and independent variables.
    reg_lst: list
        A list containing multiple lists of dependent variable and independent
        variables, e.g., [['Y', 'X1', ...],..., ['Y', 'X1', ...,]].
    reg_order: list
        The order of independent variables in result table.
    reg_names: list
        The names for each regression.
    params_format: str
        The number of decimal places for parameters, e.g., '{:.3f}'.
    tvalues_format: str
        The number of decimal places for t-values, e.g., '{:.2f}'.
    '''

    # Create a DataFrame
    rows = sum([[var, f'{var}_t'] for var in ['const'] + reg_order], [])
    if reg_names is None:
        reg_names = [f'({i+1})' for i in range(len(reg_lst))]
    show = pd.DataFrame(index=rows, columns=reg_names)

    var_used = []
    
    for reg, reg_name in zip(reg_lst, reg_names):
        df = DF.loc[:, reg].copy().dropna()
        T = len(df.index.get_level_values(df.index.names[1]).unique())
#         lag = math.floor(4*(T/100)**(2/9))
        lag = 6
        fmb = FamaMacBeth(df[reg[0]], sm.add_constant(df[reg[1:]]))
        # Newey-West adjust
        fmb = fmb.fit(cov_type='kernel', bandwidth=lag)

        # params, tvalues(tstats) and pvalues
        params = fmb.params
        tvalues = fmb.tstats
        pvalues = fmb.pvalues

        # Obs.
#         total_obs = fmb.nobs
        mean_obs = int(fmb.time_info['mean'])

        # average rsquared_adj
        dft = df.reset_index(level=df.index.names[0], drop=True).copy()
        rsquared_adj = []
        for month in dft.index.unique():
            dftm = dft.loc[month].copy()
            ols = sm.OLS(dftm[reg[0]], sm.add_constant(dftm[reg[1:]])).fit()
            rsquared_adj.append(ols.rsquared_adj)
        ar2a = np.mean(rsquared_adj)

        # params and significance
        ps_lst = []
        for param, pvalue in zip(params, pvalues):
            param = params_format.format(param)
            if (pvalue <= 0.1) & (pvalue > 0.05):
                param = param + '*'
            elif (pvalue <= 0.05) & (pvalue > 0.01):
                param = param + '**'
            elif pvalue <= 0.01:
                param = param + '***'
            ps_lst.append(param)

        # params and tvalues
        tvalues = [tvalues_format.format(t) for t in tvalues]
        t_lst = [f'({t})' for t in tvalues]
        pt_lst = [[i, j] for i, j in zip(ps_lst, t_lst)]

        # put them in place
        for var, pt in zip(['const'] + reg[1:], pt_lst):
            show.loc[var, reg_name] = pt[0]
            show.loc[f'{var}_t', reg_name] = pt[1]
            if((var not in var_used) and (var != 'const')):
                var_used.append(var)
#         show.loc['No. Obs.', reg_name] = str(total_obs)
        show.loc['Avg. Obs.', reg_name] = str(mean_obs)
        show.loc['Avg. Adj. R²', reg_name] = '{:.4f}%'.format(ar2a * 100)

    rename_index = sum([[var, ''] for var in ['Intercept'] + reg_order + var_used], [])
    tmp = list(show.index)
    tmp.remove('Avg. Obs.')
    tmp.remove('Avg. Adj. R²')
    show = show.reindex(index=tmp+['Avg. Obs.','Avg. Adj. R²'])

    show.index = rename_index + ['Avg. Obs.', 'Avg. Adj. R²']

    return show.dropna(axis=0, how='all').fillna('')

In [21]:
FamaMacBeth_summary(df_dropna, reg_lst=[['Ret_Next','RetDiff'],['Ret_Next','Cret','Lret']],
                    reg_order=[],reg_names = ['1','3']).to_csv('T3FMReg.csv')

In [22]:
df_dropna.sort_index(level=1).sort_index(level=0).to_csv('forT3FMReg.csv')

In [39]:
FamaMacBeth_summary(df_dropna, reg_lst=[['Ret_Next','RetDiff'], ['Ret_Next','RetDiff','MktCap'],
                                        ['Ret_Next','Cret','Lret'], ['Ret_Next','Cret','Lret', 'MktCap'],
                                        ['Ret_Next','Cret', 'MktCap'], ['Ret_Next','Lret','MktCap']],
                    reg_order=[],reg_names = ['1','2','3','4','5','6']).to_csv('T3FMReg.csv')